# Load Required Packages

In [ ]:
!pip install pygris
!pip install matplotlib

import pandas as pd
import geopandas as gpd

# Start Accessbility Analysis

In [ ]:
from pygris import tracts
import matplotlib.pyplot as plt

# Define CapMetro counties
capmetro_counties = ["Travis", "Williamson", "Hays", "Bastrop", "Caldwell", "Burnet"]

# Pull 2023 tracts
capmetro_tracts = tracts(state="TX", county=capmetro_counties, cb=True, cache=True, year=2023)

# Plot
capmetro_tracts.plot(figsize=(10, 10), edgecolor='black')
plt.title("2023 Census Tracts in CapMetro Service Area")
plt.axis("off")
plt.show()

In [ ]:
# Copy the tract geometry
capmetro_centroids = capmetro_tracts.copy()

# Project to Texas Central (EPSG:3081) for accurate centroid computation
projected = capmetro_tracts.to_crs("EPSG:3081")

# Compute centroids in projected space, then convert back to original CRS
capmetro_centroids['geometry'] = projected.centroid.to_crs(capmetro_tracts.crs)

# Inspect
capmetro_centroids.head()

In [ ]:
import geopandas as gpd

# Create a GeoDataFrame with only centroid geometries and tract IDs
origins = gpd.GeoDataFrame(
    geometry=capmetro_centroids.geometry,
    data={'id': capmetro_centroids['GEOID']},
    crs=capmetro_centroids.crs
)

# Reset index for clean export/use
origins.reset_index(drop=True, inplace=True)

# Preview
origins.head()

In [ ]:
from pygris.data import get_census

# Define ACS variables for race/ethnicity
race_vars = {
    "B03002_001E": "total",   # Total population
    "B03002_003E": "white",   # White alone, not Hispanic or Latino
    "B03002_004E": "black",   # Black or African American alone
    "B03002_012E": "latino"   # Hispanic or Latino
}

# Pull ACS 5-Year 2023 estimates for all Texas tracts
tx_race = get_census(
    dataset="acs/acs5",
    variables=list(race_vars.keys()),
    year=2023,
    params={
        "for": "tract:*",
        "in": "state:48"  # 48 = Texas
    },
    guess_dtypes=True,
    return_geoid=True
)

# Rename columns using the race_vars dictionary
tx_race.rename(columns=race_vars, inplace=True)

# Preview the cleaned data
tx_race.head()

CapMetro expanded MetroRapid started service on February 23, 2025 with two more routes: The Pleasant Valley Line (Route 800) and the Expo Center Line (Route 837). Route 800 runs from Barbara Jordan to Vertex & Slaughter (with a future expansion to Goodnight Ranch Park & Ride) primarily along Pleasant Valley Road, supplementing Routes 7, 300 and 350 on portions of the route. Route 837 runs from Republic Square to Loyola & Decker (with a future expansion to Travis County Expo Center) primarily along Manor Road and supplementing Routes 20 and 337 on most of the route

# New Travel Times -- April X, 2025

In [ ]:
from r5py import TransportNetwork

# Replace with your Austin .pbf file and CapMetro GTFS file
transport_network_new = TransportNetwork(
    "openstreetmap_fr_central-latest.osm.pbf",
    ["DataPortal/gtfs_new.zip"]
)

In [ ]:
import datetime
import r5py

# Create the travel time matrix for Austin with relevant parameters
travel_times_new = r5py.TravelTimeMatrix(
    transport_network=transport_network_new,
    origins=origins,  # must be GeoDataFrame with 'geometry' and 'id' column
    destinations=origins,
    max_time=datetime.timedelta(hours=2),
    departure=datetime.datetime(2025, 4, 1, 7, 0),  # date after MetroRapid 800/837 launch
    departure_time_window=datetime.timedelta(hours=2),
    transport_modes=["TRANSIT", "WALK"]
)



In [ ]:
travel_times_new.to_csv("output/transit_time_matrix_7am_April1_2025.csv", index=False)

In [ ]:
travel_times_new.head()

In [ ]:
travel_times_new.travel_time.describe()

# Quality Check

In [ ]:
# Filter for Travis County (COUNTYFP = '453')
travis_tracts = capmetro_tracts[capmetro_tracts['COUNTYFP'] == '453'].to_crs('EPSG:4326')

# Compute centroids using projected CRS for spatial accuracy, then convert back to WGS84
travis_centroids = travis_tracts.to_crs('EPSG:6583').centroid.to_crs('EPSG:4326')

# Create GeoDataFrame with centroids
travis_origins = gpd.GeoDataFrame(geometry=travis_centroids)
travis_origins['id'] = travis_origins.index
travis_origins = travis_origins.reset_index(drop=True)

# Sample 50 points for quality check
travis_origins_sample = travis_origins.sample(n=50, random_state=42).reset_index(drop=True)

# Preview
travis_origins_sample.head()

In [ ]:
from shapely.geometry import Point

AUSTIN_DOWNTOWN_STATION = Point(-97.7394, 30.2653)

import pandas as pd

austin_downtown_station = pd.DataFrame({
    'longitude': [-97.7394],
    'latitude': [30.2653]
})

austin_downtown_station

In [ ]:
# Create destination GeoDataFrame
destinations = gpd.GeoDataFrame(
    {
        "id": [1],
        "geometry": [AUSTIN_DOWNTOWN_STATION]
    },
    crs="EPSG:4326"
)

# Run detailed itineraries
detailed_itineraries = r5py.DetailedItineraries(
    transport_network=transport_network_new,
    origins=travis_origins_sample,
    destinations=destinations,
    departure=datetime.datetime(2025, 4, 1, 7, 0),  # post Project Connect 800/837 launch
    max_time=datetime.timedelta(hours=1),
    transport_modes=[r5py.TransportMode.TRANSIT, r5py.TransportMode.WALK],
    snap_to_network=True
)
detailed_itineraries 

In [ ]:
detailed_itineraries.transport_mode.value_counts()

# Old Travel Times -- June , 2017

In [ ]:
# Replace with your Austin .pbf file and CapMetro GTFS file
transport_network_before = TransportNetwork(
    "openstreetmap_fr_central-latest.osm.pbf",
    ["DataPortal/gtfs-2017-06-01.zip"]
)

In [ ]:
# Create the travel time matrix for Austin with relevant parameters
travel_times_before = r5py.TravelTimeMatrix(
    transport_network=transport_network_before,
    origins=origins,  # must be GeoDataFrame with 'geometry' and 'id' column
    destinations=origins,
    max_time=datetime.timedelta(hours=2),
    departure=datetime.datetime(2017, 6, 16, 7, 0),  
    departure_time_window=datetime.timedelta(hours=2),
    transport_modes=["TRANSIT", "WALK"]
)

In [ ]:
travel_times_before.to_csv("output/transit_time_matrix_7am_June16_2017.csv", index=False)

In [ ]:
travel_times_before.travel_time.describe()

In [ ]:
#Quality check
# Create destination GeoDataFrame
destinations = gpd.GeoDataFrame(
    {
        "id": [1],
        "geometry": [AUSTIN_DOWNTOWN_STATION]
    },
    crs="EPSG:4326"
)
# Run detailed itineraries
detailed_itineraries = r5py.DetailedItineraries(
    transport_network=transport_network_before,
    origins=travis_origins_sample,
    destinations=destinations,
    departure=datetime.datetime(2017, 6, 16, 7, 0),  # post Project Connect 800/837 launch
    max_time=datetime.timedelta(hours=1),
    transport_modes=[r5py.TransportMode.TRANSIT, r5py.TransportMode.WALK],
    snap_to_network=True)

In [ ]:
detailed_itineraries.transport_mode.value_counts()

# Process LODES Data

In [ ]:

# Load WAC data for Texas (2022)
wac = pd.read_csv("LODES/tx_wac_S000_JT00_2022.csv", dtype={"w_geocode": "str"})
wac.head()

In [ ]:
wac['tract_id'] = wac['w_geocode'].str[:11]
wac['tract_id'] = wac['tract_id'].astype(str)

In [ ]:
wac_tract = wac.groupby('tract_id').agg({'C000': 'sum'}).reset_index()
wac_tract

In [ ]:
# Count number of NAs in the dataset
print(wac_tract['C000'].isna().sum())

# LODES Alternative

In [ ]:
from pygris.data import get_lodes

tx_lodes_wac22 = get_lodes(
  state = "TX", 
  year = 2022, 
  lodes_type = "wac",
  cache = True,
  return_lonlat = True
)

tx_lodes_wac17 = get_lodes(
  state = "TX", 
  year = 2017, 
  lodes_type = "wac",
  cache = True,
  return_lonlat = True
)

In [ ]:
tx_lodes_wac22.head()

In [ ]:
tx_lodes_wac17.head()

In [ ]:
tx_lodes_wac22['tract_id'] = tx_lodes_wac22['w_geocode'].str[:11]
tx_lodes_wac22

tx_lodes_wac22.to_csv("output/tx_lodes_wac22.csv", index=False)

In [ ]:
tx_lodes_wac17['tract_id'] = tx_lodes_wac17['w_geocode'].str[:11]
tx_lodes_wac17
tx_lodes_wac17.to_csv("output/tx_lodes_wac17.csv", index=False)

In [ ]:
tx_wac22 = tx_lodes_wac22.groupby('tract_id').agg({'C000': 'sum'}).reset_index()
tx_wac22

In [ ]:
tx_wac17 = tx_lodes_wac17.groupby('tract_id').agg({'C000': 'sum'}).reset_index()
tx_wac17

# Accessibility Before and After

In [ ]:
# Filter on travel time threshold 
travel_times_before = travel_times_before[travel_times_before['travel_time'] < 45]
travel_times_new = travel_times_new[travel_times_new['travel_time'] < 45]

travel_times_before

In [ ]:
travel_times_new

In [ ]:
# Merge in jobs
acc_before = travel_times_before.merge(tx_wac17, how = 'left', left_on = 'to_id', right_on = 'tract_id')
acc_new = travel_times_new.merge(tx_wac22, how = 'left', left_on = 'to_id', right_on = 'tract_id')
acc_before


In [ ]:
# Merge in jobs
acc_before = travel_times_before.merge(tx_wac17, how = 'left', left_on = 'to_id', right_on = 'tract_id')
acc_new = travel_times_new.merge(tx_wac22, how = 'left', left_on = 'to_id', right_on = 'tract_id')
acc_before

In [ ]:
acc_new 

In [ ]:
# Create summaries
acc_before_final = acc_before.groupby('from_id').agg({'C000': 'sum'}).reset_index()
acc_new_final = acc_new.groupby('from_id').agg({'C000': 'sum'}).reset_index()

In [ ]:
acc_before_final

In [ ]:
acc_new_final

In [ ]:
acc_comparison = acc_before_final.merge(acc_new_final, how = 'outer', on = 'from_id', suffixes = ('_before', '_new'))
acc_comparison

In [ ]:
acc_comparison['diff'] = acc_comparison['C000_new'] - acc_comparison['C000_before']

acc_comparison['diff'].describe()

In [ ]:
austin_tracts_final = capmetro_tracts.merge(acc_comparison, how = 'left', left_on = 'GEOID', right_on = 'from_id')
austin_tracts_final

austin_tracts_final.to_csv("output/austin_tracts_final.csv", index=False)

In [ ]:
austin_tracts_final

In [ ]:
import altair as alt

# Filter just in case austin_tracts_final contains more than central Austin
chart_data = austin_tracts_final  # Replace or subset if needed

alt.Chart(chart_data).mark_geoshape(
    stroke='white',
    strokeWidth=0.2
).encode(
    color=alt.Color('C000_before:Q',
                    scale=alt.Scale(scheme='viridis',reverse=True),
                    title="Jobs Accessible (2017)"),
    tooltip=['GEOID:N', 'C000_before:Q']
).project(
    type='mercator',  # More reliable for small areas than 'albersUsa'
    center=[-97.7431, 30.2672],  # Downtown Austin
    scale=50000  # Increased zoom
).properties(
    width=600,
    height=400,
    title="Central Austin Job Accessibility (2017)"
)


In [ ]:
import altair as alt

# Filter just in case austin_tracts_final contains more than central Austin
chart_data = austin_tracts_final  # Replace or subset if needed

alt.Chart(chart_data).mark_geoshape(
    stroke='white',
    strokeWidth=0.2
).encode(
    color=alt.Color('C000_new:Q',
                    scale=alt.Scale(scheme='viridis',reverse=True),
                    title="Jobs Accessible (2022)"),
    tooltip=['GEOID:N', 'C000_new:Q']
).project(
    type='mercator',  # More reliable for small areas than 'albersUsa'
    center=[-97.7431, 30.2672],  # Downtown Austin
    scale=50000  # Increased zoom
).properties(
    width=600,
    height=400,
    title="Central Austin Job Accessibility (2022)"
)

In [ ]:
import altair as alt

alt.Chart(austin_tracts_final).mark_geoshape().encode(
    color=alt.Color(
        'diff:Q',
        scale=alt.Scale(
            scheme='redblue',
            domain=[-5000, 250000],  # Full range including negatives
            domainMid=0                # White at 0
        ),
        title="Change in Job Access"
    ),
    tooltip=['GEOID:N', 'diff:Q']
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300,
    title="Accessibility Change (with Negative Values in Legend)"
)

In [ ]:
# If using FIPS code for Travis County
travis_tracts = austin_tracts_final[austin_tracts_final['COUNTYFP'] == '453']

import altair as alt

alt.Chart(travis_tracts).mark_geoshape().encode(
    color=alt.Color('diff:Q', scale=alt.Scale(scheme='redblue')),
    tooltip=['GEOID', 'diff:Q']
).project(
    type='albersUsa'
).properties(
    width=500,
    height=400,
    title="Accessibility Change in Travis County"
)

In [ ]:
import geopandas as gpd

# Read city boundary shapefile (must be in same CRS)
austin_boundary = gpd.read_file("DataPortal/City/austin_city_boundary.shp").to_crs(austin_tracts_final.crs)

# Filter using spatial join
austin_city_tracts = gpd.sjoin(austin_tracts_final, austin_boundary, how="inner", predicate="intersects")
austin_city_tracts

In [ ]:
import altair as alt

alt.Chart(austin_city_tracts).mark_geoshape().encode(
    color=alt.Color('diff:Q', scale=alt.Scale(scheme='redblue')),
    tooltip=['GEOID_left', 'diff:Q']
).project(
    type='albersUsa'
).properties(
    width=500,
    height=400,
    title="Accessibility Change in City of Austin"
)

# Transit Accessibility Short Cut

In [ ]:
!pip install tracc

In [ ]:
import tracc

# Loading in destination data.
# Job counts by block group from the the LEHD for Texas
dfo = tracc.supply(
    supply_df =pd.read_csv("output/tx_lodes_wac22.csv"),
    columns = ["w_geocode","tract_id","C000"] # C000 pertains to the total number of jobs
    )
dfo.data.head()

In [ ]:

# Loading in travel costs.
# For this example, travel times by transit between tracts in Austin at 7am on April 1, 2025.
dft = tracc.costs(
    pd.read_csv(
    "output/transit_time_matrix_7am_April1_2025.csv")
    )
#dft.data.time = dft.data.time / 60 # converting time from seconds to minutes

dft.data.head()

In [ ]:

# Computing impedance function based on a 45 minute travel time threshold.
dft.impedence_calc(
    cost_column = "travel_time",
    impedence_func = "cumulative",
    impedence_func_params = 45,
    output_col_name = "fCij_c45",
    prune_output = False
)
dft.data.head()

In [ ]:
# Setting up the accessibility object.
# This includes joining the destination data to the travel time data.
acc = tracc.accessibility(
    travelcosts_df = dft.data,
    supply_df = dfo.data,
    travelcosts_ids = ["from_id","to_id"],
    supply_ids = "tract_id"
    )


In [ ]:


# Computing accessibility to jobs based on the 45-min threshold.
dfa = acc.potential(
    opportunity = "C000",
    impedence = "fCij_c45"
    )

dfa.head()

In [ ]:
tract_transit_acc25 = capmetro_tracts.merge(dfa, how = 'left', left_on = 'GEOID', right_on = 'from_id')
tract_transit_acc25 

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 10))

tract_transit_acc25.plot(
    column='A_C000_fCij_c45',
    ax=ax,
    scheme='quantiles',
    k=10,
    cmap='viridis',  # or 'plasma', 'YlGnBu', 'OrRd', etc.
    edgecolor='white',
    linewidth=0.3,
    legend=True,
    legend_kwds={
        'loc': 'upper right',
        'title': 'Job Access (45 min, 2025)',
        'fontsize': 8,         # smaller text
        'title_fontsize': 10   # smaller title
    }
)

# Optional: remove axis
ax.set_axis_off()

plt.title("Transit-Accessible Jobs in 2025 (CapMetro Region)", fontsize=14)
plt.tight_layout()
plt.show()